La première partie du projet sera liée à l'étude des data


In [ ]:
# def knn():
# PURITY log transform  
import pandas as pd
import numpy as np
import os 
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.model_selection import cross_val_score
import seaborn as sns
import matplotlib.pyplot as plt

def data_visualisation():
    
    # Créer un dossier pour sauvegarder les graphiques
    output_dir = 'data_visualisation'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Répertoire créé : {output_dir}")
        
    data_train = pd.read_csv('/Users/alicepriolet/Desktop/ML/epfl-bio-322-2024/train.csv')
    data_test = pd.read_csv('/Users/alicepriolet/Desktop/ML/epfl-bio-322-2024/test.csv')
    print("The size of the train data set is",data_train.shape)
    #print(data_test.shape)
    
    categorical_columns = data_train.select_dtypes(include=['object', 'category']).columns
    print("There is ",len(categorical_columns),"categorial columns")
    
        # Répartition des colonnes catégoriques et sauvegarde des graphiques
    for col in categorical_columns:
        plt.figure(figsize=(15, 15))
        sns.countplot(x=col, data=data_train, palette='viridis')
        plt.title(f"Repartition of the categories in {col}")
        plt.xlabel(col)
        plt.ylabel("Frequency")
        plt.xticks(rotation=45)
        
        # Sauvegarder le graphique
        graph_path = os.path.join(output_dir, f"category_distribution_{col}.png")
        plt.savefig(graph_path)
        plt.close()
        #print(f"Graphic saved : {graph_path}")
    
    # Identify the samples that appear several times in the data set
    duplicate_samples = data_train[data_train.duplicated(subset='sample_name', keep=False)]

    # Afficher les indices et les noms des échantillons dupliqués
    #for index, row in duplicate_samples.iterrows():
    #    print(f"Index: {index}, Sample Name: {row['sample_name']}")

    # Print the total number of duplicated samples
    print(f"Nombre total de noms de samples dupliqués : {duplicate_samples['sample_name'].nunique()}")
    print("Nombre total de samples concernés", len(duplicate_samples))
    
    
    
    # Count the number of columns of infrared spectrum measurements
    spectr_columns = [col for col in data_test.columns if col.startswith('9') or col.startswith('1')]
    spectr_count = len(spectr_columns)
    
    print(f"Number of Infrared Spectrum columns is : {spectr_count}")
    
    # Allows us to see the purity curve 
    plt.figure(figsize=(10, 6))
    sns.histplot(data_train['PURITY'], kde=True, bins=30, color='blue')
    plt.title("Distribution de la variable PURITY")
    plt.xlabel("PURITY")
    plt.ylabel("Fréquence")
    
    graph_path = os.path.join(output_dir, f"purity_frequence.png")
    plt.savefig(graph_path)
    plt.close()
    #print(f"Graphique sauvegardé : {graph_path}")

    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=data_train['908.1'], y=data_train['PURITY'], alpha=0.7)
    sns.scatterplot(x=data_train['914.3'], y=data_train['PURITY'], alpha=0.7)
    sns.scatterplot(x=data_train['920.5'], y=data_train['PURITY'], alpha=0.7)
    sns.scatterplot(x=data_train['1316.9'], y=data_train['PURITY'], alpha=0.7)
    sns.scatterplot(x=data_train['1676.2'], y=data_train['PURITY'], alpha=0.7)
    plt.title("Relation between 908.1, 914.3, 920.5. 1316.9 et 1676.2 and Purity")
    plt.xlabel("908.1, 914.3, 920.5. 1316.9 and 1676.2")
    plt.ylabel("Purity")
    
    graph_path = os.path.join(output_dir, f"relation_between_purity_and_spectrum.png")
    plt.savefig(graph_path)
    plt.close()
    print(f"Graphique sauvegardé : {graph_path}")
    

    data_train['908.1_residuals'] = data_train['908.1'] - data_train['908.1'].median()
    plt.figure(figsize=(10, 6))
    sns.histplot(data_train['908.1_residuals'], kde=True, bins=30, color='purple')
    plt.title("Distribution des résidus de 908.1")
    plt.xlabel("Résidus de 908.1")
    plt.ylabel("Fréquence")
    
    graph_path = os.path.join(output_dir, f"spectrum_distribution.png")
    plt.savefig(graph_path)
    plt.close()
    print(f"Graphique sauvegardé : {graph_path}")

    data_reduced = data_train.iloc[:, 6:]
    corr_matrix = data_reduced.corr()  # Matrice de corrélation
    plt.figure(figsize=(12, 8))
    sns.heatmap(corr_matrix, cmap='coolwarm', annot=False, cbar=True)
    plt.title("Matrice de corrélation des variables numériques")
    
    graph_path = os.path.join(output_dir, f"correlation_matrix.png")
    plt.savefig(graph_path)
    plt.close()
    print(f"Graphique sauvegardé : {graph_path}")

    top_5_correlations = (
    corr_matrix
    .abs()  # Prendre les valeurs absolues des corrélations
    .unstack()  # Convertir en série pour itérer
    .sort_values(ascending=False)  # Trier par corrélation décroissante
    .drop_duplicates()  # Supprimer les doublons
    )

    # Filtrer pour exclure les corrélations de la diagonale (valeur de corrélation = 1)
    top_5_corr = top_5_correlations[top_5_correlations < 1].head(5)

    print("voici les 5 les plus corrélés",top_5_correlations)
    
    mean_purity = data_train['PURITY'].mean()
    std_purity= data_train['PURITY'].std()
    
    print('purity mean is',mean_purity)
    print('purity std is',std_purity)
    
    # Create histogram
    sns.kdeplot(data = data_train, x = 'PURITY', fill = True)
    plt.title('Purity distribution')
    plt.xlabel('Purity')
    plt.ylabel('Frequency')
    
    graph_path = os.path.join(output_dir, f"frequency_purity.png")
    plt.savefig(graph_path)
    plt.close()
    print(f"Graphique sauvegardé : {graph_path}")
    
    # Filtrer et afficher les noms des échantillons ayant une pureté < 10
    low_purity_samples = data_train[data_train['PURITY'] < 10]['sample_name']

    # Afficher les noms des échantillons
    #for index, sample in low_purity_samples.items():
        #print(f"Index: {index}, Sample Name: {sample}")
        
    print(f"Nombre total de samples avec une pureté < 10 : {len(low_purity_samples)}")
    
    # Filtrer et afficher les noms des échantillons ayant une pureté > 60
    high_purity_samples = data_train[data_train['PURITY'] > 60]['sample_name']

    # Afficher les noms des échantillons
    for index, sample in high_purity_samples.items():
        print(f"Index: {index}, Sample Name: {sample}")
        
    print(f"Nombre total de samples avec une pureté > 60 : {len(high_purity_samples)}")
    


def preprocessing_v1(apply_one_hot=False, apply_scaling=True, apply_pca=False, apply_correlation=False, apply_remove_outliers=True):
    train_data_og = pd.read_csv('/Users/alicepriolet/Desktop/ML/epfl-bio-322-2024/train.csv')
    test_data_og = pd.read_csv('/Users/alicepriolet/Desktop/ML/epfl-bio-322-2024/test.csv')
    train_data = train_data_og.copy()
    test_data = test_data_og.copy()
    train_data = train_data.drop(columns=['prod_substance'])
    test_data = test_data.drop(columns=['prod_substance'])
    
    non_wavelength_cols = ['device_serial', 'substance_form_display', 'measure_type_display']
    wavelength_cols = train_data.columns[6:]
    
    if apply_one_hot:
        # One Hot encoding 
        encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        X_train_encoded = encoder.fit_transform(train_data[non_wavelength_cols])
        X_test_encoded = encoder.transform(test_data[non_wavelength_cols])
        
        # Convert encoded features to DataFrame
        X_train_encoded_df = pd.DataFrame(X_train_encoded, columns=encoder.get_feature_names_out(non_wavelength_cols))
        X_test_encoded_df = pd.DataFrame(X_test_encoded, columns=encoder.get_feature_names_out(non_wavelength_cols))
        
        train_data_combined = pd.concat([pd.DataFrame(X_train_encoded_df), train_data[wavelength_cols].reset_index(drop=True)], axis=1)
        test_data_combined = pd.concat([pd.DataFrame(X_test_encoded_df), test_data[wavelength_cols].reset_index(drop=True)], axis=1)
    else:
        train_data_combined = train_data
        test_data_combined = test_data
    
    # Add sample_name column back to the combined DataFrames
    train_data_combined.insert(0, 'sample_name', train_data_og['sample_name'])
    test_data_combined.insert(0, 'sample_name', test_data_og['sample_name'])
    
    # Remove NaN values
    train_data_combined = train_data_combined.dropna()
    test_data_combined = test_data_combined.dropna()
    
    y_train = train_data['PURITY'].iloc[train_data_combined.index]
    
    #NOTE: toujours mettre scaling avec remove_outliers
    
    if apply_scaling:
        # Standardize the data
        scaler = StandardScaler()
        wavelength_train_scaled = scaler.fit_transform(train_data_combined[wavelength_cols])
        wavelength_test_scaled = scaler.transform(test_data_combined[wavelength_cols])
        
        train_data_combined[wavelength_cols] = wavelength_train_scaled
        test_data_combined[wavelength_cols] = wavelength_test_scaled
    
    if apply_remove_outliers:
        # Remove outliers
        outliers_index = (np.abs(wavelength_train_scaled) > 3).any(axis=1)
        train_data_combined = train_data_combined[~outliers_index]
        y_train = y_train[~outliers_index]
        
        train_data_combined = train_data_combined.reset_index(drop=True)
        test_data_combined = test_data_combined.reset_index(drop=True)
        y_train = y_train.reset_index(drop=True)
        
    if apply_pca:
        # Perform PCA on scaled wavelength columns
        pca = PCA(n_components=5)
        wavelength_cols = train_data_combined.columns[54:]
        
        X_train_pca = pca.fit_transform(train_data_combined[wavelength_cols])
        X_test_pca = pca.transform(test_data_combined[wavelength_cols])

        # Combine PCA components with original data
        X_train_combined = pd.concat([train_data_combined.iloc[:, :54].reset_index(drop=True), 
                                      pd.DataFrame(X_train_pca, columns=[f'PC{i+1}' for i in range(5)])], axis=1)
        X_test_combined = pd.concat([test_data_combined.iloc[:, :54].reset_index(drop=True), 
                                     pd.DataFrame(X_test_pca, columns=[f'PC{i+1}' for i in range(5)])], axis=1)
        
        train_data_combined = X_train_combined
        test_data_combined = X_test_combined
        
    if apply_correlation:
    # Compute correlation matrix only for wavelength columns
        wavelength_cols = train_data_combined.columns[54:] #pq t'as mis ca
        correlation_matrix = train_data_combined[wavelength_cols].corr()

        # Visualize correlation matrix
        plt.figure(figsize=(12, 10))
        sns.heatmap(correlation_matrix, cmap='coolwarm', annot=False)
        plt.title("Correlation Matrix for Wavelength Features")
        plt.show()

        # Identify highly correlated features (e.g., |r| > 0.999)
        threshold_high = 0.999
        threshold_low = 0.2

        high_corr_pairs = [
            (i, j)
            for i in range(correlation_matrix.shape[0])
            for j in range(i + 1, correlation_matrix.shape[1])
            if abs(correlation_matrix.iloc[i, j]) > threshold_high
        ]
        
        features_to_drop = set()
        for i, j in high_corr_pairs:
            features_to_drop.add(wavelength_cols[j])  # Arbitrarily drop the second feature in the pair

        # Remove the selected features
        train_data_combined = train_data_combined.drop(columns=list(features_to_drop))
        test_data_combined = test_data_combined.drop(columns=list(features_to_drop))
        
        low_corr_pairs = [
            (i, j)
            for i in range(correlation_matrix.shape[0])
            for j in range(i + 1, correlation_matrix.shape[1])
            if abs(correlation_matrix.iloc[i, j]) < threshold_low
        ]

        print("Highly correlated features:")
        for i, j in high_corr_pairs:
            print(f"{wavelength_cols[i]} and {wavelength_cols[j]}: {correlation_matrix.iloc[i, j]}")
        print("Low correlated features:")
        for i, j in low_corr_pairs:
            print(f"{wavelength_cols[i]} and {wavelength_cols[j]}: {correlation_matrix.iloc[i, j]}")
    
    
    return train_data_combined, test_data_combined, y_train

   
def calculate_feature_importance(X_train, y_train, X_test, threshold=0.25):
    # Calculate feature importance using a linear regression model
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Print feature importance
    feature_importance = pd.Series(model.coef_, index=X_train.columns)
    feature_importance = feature_importance.abs().sort_values(ascending=False)
    wavelength_feature_importance_df = feature_importance.reset_index()
    wavelength_feature_importance_df.columns = ['Feature', 'Importance']
    wavelength_feature_importance_df.to_csv('/Users/alicepriolet/Desktop/ML/epfl-bio-322-2024/feature_importance_LR1.csv', index=False)
    print('Feature Importance saved successfully.')
    
    # Calculate stats threshold
    threshold_value = feature_importance.quantile(threshold)
    
    # Identify low-importance features
    low_importance_features = feature_importance[feature_importance < threshold_value].index
    print(f'Low importance features: {low_importance_features}')
    
    # Remove low-importance features
    X_train_reduced = X_train.drop(columns=low_importance_features)
    X_test_reduced = X_test.drop(columns=low_importance_features)
    
    return X_train_reduced, X_test_reduced

def submission_file(y_test_predicted):
    submission_reduced = pd.DataFrame({
        'ID': range(1, len(y_test_predicted) + 1),
        'PURITY': y_test_predicted
    })
    return submission_reduced


def linear_regression(feature_importance=False):
    X_train, X_test, y_train = preprocessing_v1(apply_one_hot=True, apply_correlation=True, apply_scaling=True, apply_remove_outliers=True)
    X_train = X_train.drop(columns=['sample_name'])
    X_test = X_test.drop(columns=['sample_name'])
    
    if feature_importance:
        X_train, X_test = calculate_feature_importance(X_train, y_train, X_test)
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    y_train_pred = model.predict(X_train)
    mse = mean_squared_error(y_train, y_train_pred)
    print('Training MSE:', mse)
    
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    print('CV MSE:', -cv_scores.mean())
    
    # Predict on test data
    y_test_pred = model.predict(X_test)
    
    # Create submission DataFrame
    submission = submission_file(y_test_pred)
    
    # Save submission to CSV
    submission.to_csv('/Users/alicepriolet/Desktop/ML/epfl-bio-322-2024/sample_submission_LR.csv', index=False)
    print('Submission file saved successfully.')

def polynomial_regression():
    X_train, X_test, y_train = preprocessing_v1(apply_one_hot=True, apply_scaling=True, remove_outliers=True)
    X_train = X_train.drop(columns=['sample_name'])
    X_test = X_test.drop(columns=['sample_name'])
    
    # Define the pipeline
    pipeline = Pipeline([
        ("polynomial", PolynomialFeatures()),
        ("regression", LinearRegression())
    ])
    
    # Define the parameter grid
    param_grid = {
        "polynomial__degree": np.arange(1, 4, 1)
    }
    
    # Initialize GridSearchCV
    grid_search = GridSearchCV(pipeline, param_grid, cv=10, 
                               scoring='neg_mean_squared_error', 
                               return_train_score=True)
    
    # Fit GridSearchCV
    grid_search.fit(X_train, y_train)
    
    # Get the best model
    best_model = grid_search.best_estimator_
    print("Best accuracy:", grid_search.best_score_)
    print("Best parameters:", grid_search.best_params_)
    
    # Predict on training data
    y_train_pred = best_model.predict(X_train)
    mse = mean_squared_error(y_train, y_train_pred)
    print('Training MSE:', mse)
    
    # Cross-validation score
    cv_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    print('CV MSE:', -cv_scores.mean())
    
    # Predict on test data
    y_test_pred = best_model.predict(X_test)
    
    # Create submission DataFrame
    submission = submission_file(y_test_pred)
    
    # Save submission to CSV
    submission.to_csv('/Users/alicepriolet/Desktop/ML/epfl-bio-322-2024/sample_submission_POLY.csv', index=False)
    print('Submission file saved successfully.')


def logistic_regression_with_preprocessed_data():
    """
    Implémente une régression logistique sur des données prétraitées
    avec preprocessing_v1. Évalue le modèle et génère un fichier
    de soumission.
    """
    # Appliquer le prétraitement
    X_train, X_test, y_train = preprocessing_v1(
        apply_one_hot=True,
        apply_scaling=True,
        apply_remove_outliers=True,
        apply_pca=False,  # Désactiver PCA pour garder toutes les dimensions
        apply_correlation=False  # Désactiver le nettoyage par corrélation
    )

    # Supprimer la colonne 'sample_name' inutilisée
    X_train = X_train.drop(columns=['sample_name'], errors='ignore')
    X_test = X_test.drop(columns=['sample_name'], errors='ignore')

    # Binariser la cible pour une tâche de classification binaire
    threshold = 0.5  # Seuil pour séparer les classes
    y_train_binary = (y_train > threshold).astype(int)  # 0 si ≤ seuil, 1 si > seuil

    # Initialiser et entraîner le modèle de régression logistique
    model = LogisticRegression(max_iter=1000, solver='lbfgs')
    model.fit(X_train, y_train_binary)

    # Évaluer les performances sur les données d'entraînement
    y_train_pred = model.predict(X_train)
    train_accuracy = accuracy_score(y_train_binary, y_train_pred)
    print("Training Accuracy:", train_accuracy)
    print("\nClassification Report (Training):")
    print(classification_report(y_train_binary, y_train_pred))

    # Calculer l'AUC (Area Under Curve)
    y_train_proba = model.predict_proba(X_train)[:, 1]
    auc = roc_auc_score(y_train_binary, y_train_proba)
    print(f"Training AUC: {auc}")

    # Prédictions sur l'ensemble de test
    y_test_pred = model.predict_proba(X_test)[:, 1]  # Probabilités pour la classe positive

    # Générer le fichier de soumission
    submission = submission_file(y_test_pred)
    output_dir = '/Users/alicepriolet/Desktop/ML/epfl-bio-322-2024'
    submission_path = os.path.join(output_dir, 'sample_submission_Logistic.csv') 
    submission.to_csv(submission_path, index=False)
    print(f"Submission file saved successfully ")
    
    

def main():
    data_visualisation()
    #linear_regression(feature_importance=True)
    #polynomial_regression()
    #logistic_regression_with_preprocessed_data()
    #inspect_data_grouped_and_save('/Users/alicepriolet/Desktop/ML/epfl-bio-322-2024/train.csv')
    
 
   

if __name__ == '__main__':
    main() 

The size of the train data set is (1300, 131)
There is  5 categorial columns


/var/folders/7x/43mgl97d0fvdqdc5rb7fhwdc0000gn/T/ipykernel_76084/3215523143.py:39: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(x=col, data=data_train, palette='viridis')
/var/folders/7x/43mgl97d0fvdqdc5rb7fhwdc0000gn/T/ipykernel_76084/3215523143.py:39: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(x=col, data=data_train, palette='viridis')
/var/folders/7x/43mgl97d0fvdqdc5rb7fhwdc0000gn/T/ipykernel_76084/3215523143.py:39: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(x=col, data=data_train, palette='viridis')
/var/folders/7x/43mgl97d0

Nombre total de noms de samples dupliqués : 234
Nombre total de samples concernés 640
Number of Infrared Spectrum columns is : 125
Graphique sauvegardé : data_visualisation/relation_between_purity_and_spectrum.png
Graphique sauvegardé : data_visualisation/spectrum_distribution.png
Graphique sauvegardé : data_visualisation/correlation_matrix.png
voici les 5 les plus corrélés 908.1_residuals  908.1     1.000000
908.1            908.1     1.000000
1162.1           1168.3    0.999988
1316.9           1310.7    0.999984
1397.5           1391.3    0.999976
                             ...   
1248.8           1670      0.507092
1676.2           1242.6    0.506866
1645.2           1236.4    0.506376
1670             1242.6    0.505995
1242.6           1645.2    0.505936
Length: 7844, dtype: float64
purity mean is 27.02156252675692
purity std is 15.67221912592189
Graphique sauvegardé : data_visualisation/frequency_purity.png
Nombre total de samples avec une pureté < 10 : 85
Index: 244, Sample N